# DISASTER RESPONSE PIPELINE

## ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import sys
import re
from sqlalchemy import create_engine
import pickle


# import relevant functions/modules from the sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
database_filepath = 'DisasterResponse.db'
engine = create_engine('sqlite:///'+ database_filepath)
df_data = pd.read_sql_table('DisasterResponse', engine)

# Remove child alone as it has all zeros only
df_data = df_data.drop(['child_alone'],axis=1)

# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class
df_data['related']=df_data['related'].map(lambda x: 1 if x == 2 else x)

X = df_data.message
y = df_data[df_data.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # detect all URL present in the messages
    detected_urls = re.findall(url_regex, text)
    
    # replace URL with "urlplaceholder"    
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
                
    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words
    stop_words = stopwords.words("english")
    tokens = [tok for tok in tokens if tok not in stop_words]
     
    #Lemmanitizer to remove inflectional and derivationally related forms of a word    
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('cvect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test) # predict
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report
print(class_report)
print('Accuracy: {}'.format(np.mean(y_test.values == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       0.87      0.91      0.89      4046
               request       0.74      0.61      0.67       892
                 offer       0.00      0.00      0.00        28
           aid_related       0.73      0.68      0.71      2199
          medical_help       0.58      0.26      0.36       409
      medical_products       0.67      0.26      0.37       279
     search_and_rescue       0.73      0.14      0.24       154
              security       0.57      0.04      0.07       111
              military       0.56      0.29      0.38       171
                 water       0.73      0.60      0.66       345
                  food       0.83      0.69      0.76       598
               shelter       0.75      0.53      0.62       447
              clothing       0.80      0.38      0.52        87
                 money       0.64      0.23      0.34       125
        missing_people       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f7ed6bbe730>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
             n_jobs=1),
              n_jobs=1))],
 'cvect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
       

In [8]:
#model parameters for GridSearchCV
parameters = {'cvect__ngram_range': ((1, 1), (1, 2)),
                  'cvect__max_df': (0.75, 1.0)
                  }

cv = GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 1) ...................
[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 1), score=0.2792161350307538, total=  17.3s
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 1) ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.5s remaining:    0.0s


[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 1), score=0.27320841081390357, total=  17.3s
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 1) ...................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   54.9s remaining:    0.0s


[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 1), score=0.2822603719599428, total=  17.4s
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 2) ...................
[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 2), score=0.2773566013445859, total=  21.2s
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 2) ...................
[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 2), score=0.28107566871692174, total=  21.0s
[CV] cvect__max_df=0.75, cvect__ngram_range=(1, 2) ...................
[CV]  cvect__max_df=0.75, cvect__ngram_range=(1, 2), score=0.28526466380543636, total=  21.0s
[CV] cvect__max_df=1.0, cvect__ngram_range=(1, 1) ....................
[CV]  cvect__max_df=1.0, cvect__ngram_range=(1, 1), score=0.2780718066084966, total=  17.6s
[CV] cvect__max_df=1.0, cvect__ngram_range=(1, 1) ....................
[CV]  cvect__max_df=1.0, cvect__ngram_range=(1, 1), score=0.27406665713059647, total=  17.5s
[CV] cvect__max_df=1.0, cvect__ngram_range=(1, 1) ....................
[CV]  cvect__ma

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.9min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'cvect__ngram_range': ((1, 1), (1, 2)), 'cvect__max_df': (0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [10]:
# Get the prediction values from the grid search cross validator
y_test = cv.predict(X_test)
y_train = cv.predict(X_train)

In [11]:
#finding the best paramesters based on grip search
print(cv.best_params_)

{'cvect__max_df': 0.75, 'cvect__ngram_range': (1, 2)}


In [12]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('cvect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        str...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
class_report = classification_report(y_test, y_pred, target_names=category_names) # print classification report
print(class_report)

#print('Accuracy: {}'.format(np.mean(y_test.values == y_pred))) # print accuracy score

                        precision    recall  f1-score   support

               related       0.98      0.94      0.96      4406
               request       0.89      0.83      0.86       785
                 offer       0.00      0.00      0.00         0
           aid_related       0.93      0.82      0.87      2339
          medical_help       0.77      0.70      0.73       207
      medical_products       0.80      0.77      0.79       111
     search_and_rescue       0.83      0.86      0.85        29
              security       0.14      1.00      0.25         1
              military       0.81      0.83      0.82        88
                 water       0.92      0.78      0.84       331
                  food       0.95      0.85      0.90       553
               shelter       0.89      0.83      0.86       341
              clothing       0.98      0.85      0.91        47
                 money       0.71      0.70      0.70        46
        missing_people       0.83      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.